In [ ]:
library(xgboost)
library(tidyverse)
library(caret)
library(e1071)
library(dplyr)
library(iml)

data <- read.csv("## Provide data path here ##")
seed_values <- c(124,125,126)

*** SVM Model ***

In [ ]:
data <- read.csv("## Provide data path here ##")

seed_values <- c(124,125,126)   # Seed values for reproducibility
kernel_type <- "radial"         # Define the SVM kernel type (can be changed)

# Create an empty data frame to store misclassified sample information
df_results_misclassified <- data.frame()

# Loop through each seed value to train and test the SVM models
for (seed in seed_values) {
  print(seed)  # Display the current seed value
  set.seed(seed)  # Set the seed for reproducibility
  
  # Create stratified folds for cross-validation
  folds <- createFolds(factor(data$Label), k = 5) 

  # Initialize an empty data frame to store performance results
  df_results <- data.frame()

  # Iterate through each fold to train and evaluate the model
  for (fold_index in 1:length(folds)) {
    fold_indice_name <- names(folds[fold_index])  # Get the name of the current fold

    # Get the testing set indices for the current fold
    testing_indices <- folds[[fold_indice_name]]

    # Create a dataframe for the test samples (sample ID and labels)
    testing_samples <- data[testing_indices, names(data) %in% c("Sample","Label")]
    rownames(testing_samples) <- NULL  # Reset rownames for testing samples

    # Prepare training data excluding the testing indices
    model_trainingset <- data[-testing_indices, ]
    model_trainingset_labels <- model_trainingset$Label  # Extract training labels
    model_trainingset <- model_trainingset[,!names(model_trainingset) %in% c("Sample","Label")]  # Remove sample ID and label columns

    # Hyperparameter Tuning for SVM
    tc = tune.control(cross = 5)  # Use 5-fold cross-validation for tuning
    svm_tune = tune(
      svm, train.x = model_trainingset, train.y = model_trainingset_labels,
      kernel = kernel_type,           # Specify kernel type (e.g., radial)
      tunecontrol = tc,               # Use the defined tune control
      ranges = list(                  # Define ranges for cost and gamma parameters
        cost = c(seq(7, 9, by = 0.025)),
        gamma = outer(1:9, 10^(-4:-5)) %>% t() %>% as.vector()
      )
    )

    # Extract the best hyperparameters from the tuning results
    best_cost = svm_tune$best.parameters$cost
    best_gamma = svm_tune$best.parameters$gamma

    # Train the SVM model with the best hyperparameters
    svm_model = svm(
      x = model_trainingset, y = model_trainingset_labels, 
      type = "nu-classification",      # Specify SVM type
      kernel = kernel_type,            # Use the specified kernel
      probability = TRUE,              # Enable probability estimates
      gamma = best_gamma,              # Use the best gamma parameter
      cost = best_cost,                # Use the best cost parameter
      cross = 0,                       # No additional cross-validation
      scale = F                        # Do not scale features
    )

    # Prepare the testing set for evaluation
    model_testingset <- data[testing_indices, ]
    model_testingset_labels <- data.matrix(model_testingset$Label)  # Convert labels to matrix format
    model_testingset <- model_testingset[,!names(model_testingset) %in% c("Sample","Label")]  # Remove sample ID and label columns

    # Predict the testing set using the trained SVM model
    cat("\n RESULTS ON TESTING SET : \n")
    class_model_pred <- predict(svm_model, model_testingset, type = "raw")  # Generate predictions
  }
}


*** XGB Model ***

In [ ]:
# Create an empty data frame to store misclassified results
df_results_misclassified <- data.frame()

for (seed in seed_values) {
	set.seed(seed)

	folds <-  createFolds(factor(data$Label), k = 5) #Create 5 folds - stratified
	df_results <- data.frame()
    
	for (fold_index in 1:length(folds)) {

		fold_indice_name <-  names(folds[fold_index])  

		testing_indices <-  folds[[fold_indice_name]] # Indices in data
		model_filename <-  paste0(fold_indice_name,"_XGBoost_ClasseHealthyvsHFOnly_Model.rds") 

		testing_samples <-data[testing_indices, names(data) %in% c("Sample","Label")] #Get samples id + label
		rownames(testing_samples) <- NULL #remove wrong rownames id

		fold_filename <- paste0(fold_indice_name,"_XGBoost_ClasseHealthyvsHFOnly_FoldSamplesList_TestingSet_Seed",seed,".csv") 
		
		model_trainingset <-  data[-testing_indices, ] # Make training set from the testing folds indices
		model_trainingset_labels <-  data.matrix(model_trainingset$Label) # True labels from the training set as matrix
		model_trainingset <-  model_trainingset[,!names(model_trainingset) %in% c("Sample","Label")] # Remove Sample id and Labels from the training set
		model_trainingset <-  data.matrix(model_trainingset) # Transform dataframe as matrix so it can be used with xgb library

		XGB_dtrain <-  xgb.DMatrix(data = model_trainingset, label= model_trainingset_labels) # Object from XGB library.


		xgb_grid_1 <-  expand.grid(
			nrounds = 1000,
			eta = c(0, 0.2, 0.4, 0.6, 0.8, 1.0),
			max_depth = c(1,5,10,15,20),
			gamma = c(0,1,2,5,10),               #default=0
			colsample_bytree = 1,    #default=1 step=0.1
			min_child_weight = c(0,0.5,1,2,5),     #default=1, step=0.1, Intuitively, this is the minimum number of samples that a node can represent in order to be split further. If there are fewer than min_child_weight samples at that node, the node becomes a leaf and is no longer split. This can help reduce the model complexity and prevent overfitting.
			subsample = 1    #default=1
		)

		xgb_trcontrol_1 <-  trainControl(
			method = "cv",
			number = 5,
			verboseIter = FALSE,
			returnData = FALSE,
			returnResamp = "all",                                                        # save losses across all models
			classProbs = TRUE,                                                           # set to TRUE for AUC to be computed
			allowParallel = TRUE
		)

# Train the model : XGBoost
		xgb_train_1 <-  train(
			x = model_trainingset,
			y = make.names(model_trainingset_labels),
			trControl = xgb_trcontrol_1,
			tuneGrid = xgb_grid_1,
			method = "xgbTree"
		)

		best_nrounds <-  xgb_train_1$bestTune$nrounds 
		best_max_depth <-  xgb_train_1$bestTune$max_depth
		best_eta <-  xgb_train_1$bestTune$eta
		best_gamma <-  xgb_train_1$bestTune$gamma
		best_colsample <-  xgb_train_1$bestTune$colsample_bytree
		best_min_child <-  xgb_train_1$bestTune$min_child_weight
		best_subsample <-  xgb_train_1$bestTune$subsample


		df_hp <- data.frame(nround= best_nrounds,
															max_depth=best_max_depth,
															eta=best_eta,
															gamma=best_gamma,
															colsample=best_colsample,
															min_child=best_min_child,
															sub_sample=best_subsample)
	

		xgb_model <-  xgboost(data = XGB_dtrain, # the data   
													nrounds = best_nrounds,# max number of boosting iterations
													eta = best_eta, #learning rate 0.001
													max_depth = best_max_depth, #max deppth of trees
													gamma =best_gamma,               #default=0
													colsample_bytree = best_colsample,    #default=1, is the subsample ratio of columns when constructing each tree. Subsampling occurs once for every tree constructed.
													min_child_weight = best_min_child,     #default=1, Intuitively, this is the minimum number of samples that a node can represent in order to be split further. If there are fewer than min_child_weight samples at that node, the node becomes a leaf and is no longer split. This can help reduce the model complexity and prevent overfitting.
													subsample = best_subsample,     #default=1
													early_stopping_rounds = 5,
													objective = "binary:logistic"
		) 


		model_testingset <-  data[testing_indices, ] #Make training set from the testing folds indices.
		model_testingset_labels <-  data.matrix(model_testingset$Label) #True labels of testing set
		model_testingset <-  model_testingset[,!names(model_testingset) %in% c("Sample","Label")] #Remove labels and ids
		model_testingset <-  data.matrix(model_testingset) #transform dataframe as matrix so it can be used with xgb library

		XGB_dtest <- xgb.DMatrix(data = model_testingset, label= model_testingset_labels)
		model_pred <-  predict(xgb_model, XGB_dtest,type="response") #prediction of the testing set using corresponding XGB model

		xgb_pred_class = as.numeric(model_pred > 0.50)
		xgb_pred_class

		

		}

}

*** LIME ***

In [ ]:
LIME_output_all <-  function (path_dataset,seed_values,models_folder,model_kind, n_features,folder_to_save_results_files=paste0(models_folder,"LIME_Analysis/")) {

  dataset <- read.csv(path_dataset,check.names=FALSE,sep=",",colClasses=c("Label"="factor"))
  row.names(dataset) <- dataset$Sample
  
	for (seed in seed_values) {
		set.seed(seed)
		list_testingset_files <- list.files(path=".",pattern = paste0("*_TestingSet_Seed",seed,".csv"))
		
		for (i in 1:length(list_testingset_files))  {
			testingset_sample_ids_from_saved_fold_file <- read.csv(list_testingset_files[i],sep="\t")
			
			model_name <- paste0("Fold",i,"_",model_kind,"_ClasseHealthyvsHFOnly_Model.rds") 
			model <- readRDS(model_name, refhook = NULL)

				
			model_trainingset <- dataset %>% tibble::rownames_to_column("sample_ids") %>% 
				filter(!Sample %in% testingset_sample_ids_from_saved_fold_file$Sample) %>%
				tibble::column_to_rownames("sample_ids")
			model_trainingset_labels <- model_trainingset$Label
			model_trainingset <-  model_trainingset[,!names(model_trainingset) %in% c("Sample","Label")] #Remove Sample id and Labels from the training set
			
			model_testingset <- dataset %>% tibble::rownames_to_column("sample_ids") %>%
				filter(Sample %in% testingset_sample_ids_from_saved_fold_file$Sample) %>%  #Make training set from the testing folds indices. NB : use -X TO GET TRAIN SET
				tibble::column_to_rownames("sample_ids")
			model_testingset_labels <-  model_testingset$Label #True labels of testing set
			model_testingset <-  model_testingset[,!names(model_testingset) %in% c("Sample","Label")] #Remove labels and ids
			
			model_trainingset <- model_testingset 
			model_trainingset_labels <- model_testingset_labels
				
			if(model_kind=="Logit"){
				used_feature_tofit <- model$terms %>% attr(.,"dataClasses") %>% names(.)
				used_feature_tofit <- used_feature_tofit[used_feature_tofit != "Label"]
				model_testingset <- model_testingset[,names(model_testingset) %in% used_feature_tofit]
				model_trainingset <- model_trainingset[,names(model_trainingset) %in% used_feature_tofit]
			}
			
			for (n_set in 1:2)  { #2 : training_set part, 1:testing_set part
				
				explainer_model <-  lime(model_trainingset,model) #should be the training set
				
				if (n_set==1) {
					cat("\t explanation_model for testing set")
					explanation_model <-  explain(
						x =  model_testingset, 
						explainer =  explainer_model, 
						n_permutations =  5000,
						dist_fun =  "gower",
						kernel_width =  NULL, 
						n_features = n_features,#n_features, #top 10
						feature_select =  "auto",
						labels =  c("1"))
					
					
				} else {
					cat("\t explanation_model for training set")
					explanation_model <-  explain(
						x =  model_trainingset, 
						explainer =  explainer_model, 
						n_permutations =  5000,
						dist_fun =  "gower",
						kernel_width =  NULL,
						n_features = n_features,#n_features, #top 10
						feature_select =  "auto",
						labels =  c("1")) 
				}
				}

    }
  }
}

*** Feature Interaction : H-Friedman ***

In [ ]:
ia_results_list <- list()

n_iterations <- 100  # replace with the number of iterations you want to run

for (i in 1:n_iterations) {
  mod <- iml::Predictor$new(your_model,data = newdata_test,y=model_testingset_xgb_labels,predict.fun=predict.fun,class="1")
  ia <- Interaction$new(mod)

  # add a column to ia$results that indicates the iteration number
  ia$results$iteration <- i
  
  # store the ia$results in the list
  ia_results_list[[i]] <- ia$results
}

# bind all the ia$results data frames into a single data frame
interaction_df <- bind_rows(ia_results_list)
